
## **Sistema de Recomendação de Filmes com SVD**

###  **Objetivo**

Desenvolver um sistema de recomendação baseado em decomposição SVD para prever notas de usuários para filmes não avaliados, utilizando uma matriz de avaliações parcialmente preenchida.

---

### **Contexto**

Você foi contratado por uma plataforma de streaming para implementar um sistema de recomendação. A empresa possui registros das avaliações de **10 usuários** sobre **8 filmes**. No entanto, como é comum, cada usuário avaliou apenas uma fração dos filmes disponíveis.

---

###**Matriz de Avaliações**

A seguir, temos a matriz ( $R \in \mathbb{ℝ}^{10 \times 8}$ ), onde:

* As **linhas** representam usuários ( $U_1$ ) a ( $U_{10}$ )
* As **colunas** representam filmes ( $F_1$ ) a ( $F_8$ )
* Os valores são avaliações de $1$ a $5$, com **$0$** indicando ausência de avaliação

$
R =
\begin{bmatrix}
5 & 3 & 0 & 1 & 0 & 0 & 0 & 2 \\
4 & 0 & 0 & 1 & 2 & 0 & 0 & 0 \\
1 & 1 & 0 & 5 & 0 & 0 & 0 & 0 \\
0 & 0 & 5 & 4 & 0 & 0 & 0 & 0 \\
3 & 0 & 0 & 0 & 4 & 0 & 0 & 5 \\
0 & 2 & 4 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 3 & 5 & 2 \\
2 & 0 & 3 & 0 & 0 & 0 & 0 & 4 \\
0 & 0 & 0 & 0 & 0 & 5 & 4 & 0 \\
1 & 2 & 0 & 0 & 0 & 0 & 3 & 0 \\
\end{bmatrix}
$

---

###  **Tarefas**

Você deve:

1. **Pré-processar a matriz:**

   * Substituir os valores ausentes ($0$s) por alguma heurística, como:

     * a média da linha (média por usuário), ou
     * a média da coluna (média por filme)

2. **Aplicar SVD:**

   * Decompor a matriz preenchida em ( $U \Sigma V^T$ )
   * Utilizar **redução de dimensionalidade** com os ( $k = 3$ ) maiores valores singulares

3. **Reconstruir a matriz:**

   * Obter a matriz ( $\hat{R}$ ) com previsões para todas as avaliações
   * Arredondar os valores entre $1$ e $5$, se necessário

4. **Gerar recomendações:**

   * Para cada usuário, identificar os filmes com **maior avaliação prevista** que ele ainda **não avaliou**
   * Sugerir os **top $2$ filmes** para cada usuário

---

###  **Formato Esperado da Saída**

Para cada usuário:

* Avaliações previstas:
  $
  \hat{R}_{i, j} \quad \text{(matriz completa)}
  $
* Recomendações:

  Exemplo:

  ```
  Usuário 1: Recomendamos os filmes F3 (nota prevista 4.8) e F5 (4.2)
  Usuário 2: Recomendamos os filmes F2 (4.6) e F3 (4.1)
  ...
  ```

---

###  Dicas Técnicas

* Para fazer SVD em Python:

  ```python
  from numpy.linalg import svd
  U, S, VT = svd(R_filled, full_matrices=False)
  ```

* Para reconstruir com ( $k$ ) componentes:

  ```python
  k = 3
  S_k = np.diag(S[:k])
  U_k = U[:, :k]
  VT_k = VT[:k, :]
  R_hat = U_k @ S_k @ VT_k
  ```

* Para limitar os valores entre 1 e 5:

  ```python
  R_hat_clipped = np.clip(R_hat, 1, 5)
  ```





In [ ]:
import numpy as np
from numpy.linalg import svd

# Matriz de avaliações R (usuários x filmes)
R = np.array([
    [5, 3, 0, 1, 0, 0, 0, 2],
    [4, 0, 0, 1, 2, 0, 0, 0],
    [1, 1, 0, 5, 0, 0, 0, 0],
    [0, 0, 5, 4, 0, 0, 0, 0],
    [3, 0, 0, 0, 4, 0, 0, 5],
    [0, 2, 4, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 3, 5, 2],
    [2, 0, 3, 0, 0, 0, 0, 4],
    [0, 0, 0, 0, 0, 5, 4, 0],
    [1, 2, 0, 0, 0, 0, 3, 0],
], dtype=float)

# Máscara de valores observados
matrizBoolExistentes = (R != 0)

# Como há valores ausentes, utiliza-se uma alternativa de substituição, neste caso, a média
R_filled = R.copy()
for j in range(R.shape[1]):
    col = R[:, j]
    nonzero = col[col != 0]
    mean_val = np.mean(nonzero) if len(nonzero) > 0 else None
    R_filled[col == 0, j] = mean_val

print(R_filled)
U, S, VT = svd(R_filled, full_matrices=False)

k = 3
S_k = np.diag(S[:k])
U_k = U[:, :k]
VT_k = VT[:k, :]

R_hat = U_k @ S_k @ VT_k

R_hat_clipped = np.clip(R_hat, 1, 5)

print(" Recomendação do filme \n")
for i in range(R.shape[0]):
    user_id = f"Usuário {i+1}\n"
    not_rated = np.where(~matrizBoolExistentes[i])[0]
    preds = [(j, R_hat_clipped[i, j]) for j in not_rated]
    preds.sort(key=lambda x: x[1], reverse=True)
    top2 = preds[:2]
    if top2:
        recs = " - ".join([f"Filme {j+1} (nota prevista {score:.2f})\n" for j, score in top2])
        print(f"{user_id} - {recs}")
    else:
        print(f"{user_id} já avaliou todos os filmes.")



[[5.         3.         4.         1.         3.         4.
  3.25       2.        ]
 [4.         2.         4.         1.         2.         4.
  3.25       3.25      ]
 [1.         1.         4.         5.         3.         4.
  3.25       3.25      ]
 [2.66666667 2.         5.         4.         3.         4.
  3.25       3.25      ]
 [3.         2.         4.         2.75       4.         4.
  3.25       5.        ]
 [2.66666667 2.         4.         2.75       3.         4.
  1.         3.25      ]
 [2.66666667 2.         4.         2.75       3.         3.
  5.         2.        ]
 [2.         2.         3.         2.75       3.         4.
  3.25       4.        ]
 [2.66666667 2.         4.         2.75       3.         5.
  4.         3.25      ]
 [1.         2.         4.         2.75       3.         4.
  3.         3.25      ]]
 Recomendação do filme 

Usuário 1
 - Filme 3 (nota prevista 4.00)
 - Filme 6 (nota prevista 3.95)

Usuário 2
 - Filme 6 (nota prevista 3.85)
 - Film